BeeAI Framework Reasoning Patterns


🎯 Scenario: The Field Marketing Lead has asked you to help prepare their team for conference season. You create a Conference Prep Agent that uses 3 tools: web search to collect relevant news and search for up to date information, Wikipedia to provide company history and details, and the team's internal notes and artifacts.

📚 What You'll Learn

- Reasoning Patterns
- Conditional Requirements - Enforcing business logic and rules


## 🔧 Setup
First, let's install the BeeAI Framework and set up our environment.

- setting up the observability so we can capture and log the actions our agent takes
- getting the "internal documents"

In [ ]:
%pip install -Uqq arize-phoenix s3fs unstructured "requests==2.32.4"\
 "openinference-instrumentation-beeai==0.1.13" \
 "beeai-framework[duckduckgo,rag,wikipedia]" "fsspec==2025.3.0" jedi \
 "opentelemetry-api==1.37.0" "opentelemetry-sdk==1.37.0" "langgraph<=0.5.0" wikipedia

# The following wraps Notebook output
from IPython.display import HTML, display
def set_css(*_, **__):
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))
get_ipython().events.register("pre_run_cell", set_css)

Now let's import the necessary modules:


In [ ]:
import os
import asyncio
import time
import phoenix as px
import ipywidgets
from typing import Any
from datetime import date
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from beeai_framework.agents.requirement import RequirementAgent
from beeai_framework.agents.requirement.types import RequirementAgentOutput
from beeai_framework.agents.requirement.requirements import Requirement, Rule
from beeai_framework.agents.requirement.requirements.conditional import ConditionalRequirement
from beeai_framework.backend import ChatModel, ChatModelParameters
from beeai_framework.backend.document_loader import DocumentLoader
from beeai_framework.backend.embedding import EmbeddingModel
from beeai_framework.backend.text_splitter import TextSplitter
from beeai_framework.backend.vector_store import VectorStore
from beeai_framework.context import RunContext
from beeai_framework.emitter.emitter import Emitter, EventMeta
from beeai_framework.emitter.types import EmitterOptions
from beeai_framework.memory import UnconstrainedMemory
from beeai_framework.middleware.trajectory import GlobalTrajectoryMiddleware
from beeai_framework.tools import StringToolOutput, Tool, tool
from beeai_framework.tools.search.duckduckgo import DuckDuckGoSearchTool
from beeai_framework.tools.search.retrieval import VectorStoreSearchTool
from beeai_framework.tools.search.wikipedia import WikipediaTool, WikipediaToolInput
from beeai_framework.tools.think import ThinkTool
from beeai_framework.tools.weather import OpenMeteoTool
from beeai_framework.tools.types import ToolRunOptions
from openinference.instrumentation.beeai import BeeAIInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

 ## 1️⃣ LLM Providers: Choose Your AI Engine

BeeAI Framework supports 10+ LLM providers including Ollama, Groq, OpenAI, Watsonx.ai, and more, giving you flexibility to choose local or hosted models based on your needs. In this workshop we'll be working Ollama, so you will be running the model locally. You can find the documentation on how to connect to other providers [here](https://framework.beeai.dev/modules/backend).

In Colab, install and start Ollama for providing the Embedding Model.


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &

from google.colab import userdata
# Ollama - No parameters required
provider="ollama"
model="granite4:micro"
#model="granite4:tiny-h"
#model="granite3.3"
provider_model=provider+":"+model
!ollama pull $model
llm=ChatModel.from_name(provider_model, ChatModelParameters(temperature=0))


## 2️⃣ Provide a System Prompt, Memory Strategy, and Tools

In [ ]:
todays_date = date.today().strftime("%B %d, %Y")
instruct_prompt = f"""You help field marketing teams prep for conferences by answering questions on companies that they need to prepare to talk to. You produce quick and actionable briefs, doing your best to anwer the user's question.

Today's date is {todays_date}.

Tools:
- ThinkTool: Helps you plan and reason before you act. Use this tool when you need to think.
- DuckDuckGoSearchTool: Use this tool to collect current information on agendas, speakers, news, competitor moves. Include title + date + link to the resources you find in your answer. Do not use this tool for internal notes or artifacts.
- wikipedia_tool: Use this tool to get company/org history (not for breaking news). Only look up company names as the input.
- internal_document_search: past meetings, playbooks, artifacts. If you use information from this in yoour response, label it as as [Internal]. Always use this tool when internal notes or content is references.

Basic Rules:
- Be concise and practical.
- Favor recent info (agenda/news ≤30 days; exec changes/funding ≤180 days); flag older items.
- If you don't know, say so. Don't make things up.
"""

memory = UnconstrainedMemory()

### Add Tools

- The **Think tool** encourages a Re-Act pattern where the agent reasons and plans before calling a tool.

- The **DuckDuckGoSearchTool** is a Web Search tool that provides relevant data from the internet to the LLM

- We create a custom tool for searching Wikipedia


In [ ]:
think_tool = ThinkTool()

search_tool = DuckDuckGoSearchTool()

@tool
async def wikipedia_tool(query: str) -> str:
  """
  Search factual and historical information, including biography, history, politics, geography, society, culture,
  science, technology, people, animal species, mathematics, and other subjects.

  Args:
      query: The topic or question to search for on Wikipedia.

  Returns:
      The information found via searching Wikipedia.
  """
  full_text = False
  language = "en"
  tool = WikipediaTool(language=language)
  response = await tool.run(WikipediaToolInput(query=query, full_text=False))
  return response.get_text_content()

### Add a RAGTool to Search Internal Documents

In [ ]:
# Pull in an embedding model into ollama
!ollama pull nomic-embed-text:latest
embedding_model = EmbeddingModel.from_name("ollama:nomic-embed-text")

# Read the synthetic data from the public github repo
!curl --output rag_conference_prep_agent.txt https://raw.githubusercontent.com/IBM/beeai-workshop/refs/heads/main/intro_beeai_framework/rag_conference_prep_agent.txt
with open('rag_conference_prep_agent.txt', 'r') as file:
    content = file.read()

#Load the document using the DocumentLoader and split the document into chunks using the text_splitter.
loader = DocumentLoader.from_name(
    name="langchain:UnstructuredMarkdownLoader", file_path="rag_conference_prep_agent.txt"
)
try:
    documents = await loader.load()
except Exception as e:
    print(f"Failed to load documents: {e}")

# Split documents into chunks
text_splitter = TextSplitter.from_name(
    name="langchain:RecursiveCharacterTextSplitter", chunk_size=1000, chunk_overlap=200
)
try:
    documents = await text_splitter.split_documents(documents)
except Exception as e:
    print(f"Failed to split documents: {e}")
print(f"Loaded {len(documents)} document chunks")

# Create temporal vector store, which tracks time, and add documents
vector_store = VectorStore.from_name(name="beeai:TemporalVectorStore", embedding_model=embedding_model)
await vector_store.add_documents(documents=documents)
print("Vector store populated with documents")

# Create the vector store search tool. VectorStoreSearchTool is a built in tool wrapper
internal_document_search = VectorStoreSearchTool(vector_store=vector_store)

## Conditional Requirements: Guiding Agent Behavior


What Are Conditional Requirements?
[Conditional requirements](https://framework.beeai.dev/experimental/requirement-agent#conditional-requirement) ensure your agents are reliable by controlling when and how tools are used. They're like business rules for agent behavior. You can make them as strict (esentially writing a static workflow) or flexible (no rules! LLM decides) as you'd like.

The rules that you enforce may seem simple in the BeeAI framework, but in other frameworks they require ~5X the amount of code. Check out this [blog](https://beeai.dev/blog/reliable-ai-agents) where we built the same agent in BeeAI and other agent framework LangGraph.

In [ ]:
#Use widgets to choose reasoning patterns
patterns=ipywidgets.ToggleButtons(
    options=['Simple ReAct','Constrained ReAct','Planning','Unconstrained'],
    value='Simple ReAct'
    )
display(patterns)

In [ ]:
pattern = patterns.value
if pattern == 'Simple ReAct':
  requirement_set=[ConditionalRequirement(ThinkTool, force_at_step=1, force_after=Tool,consecutive_allowed=False)]
elif pattern == 'Constrained ReAct':
  requirement_set=[
    ConditionalRequirement(ThinkTool, consecutive_allowed=False, force_at_step=1 ),
    ConditionalRequirement(wikipedia_tool, only_after=ThinkTool, consecutive_allowed=True, priority=10,),
    ConditionalRequirement(DuckDuckGoSearchTool, only_after=ThinkTool, consecutive_allowed=True, min_invocations=1, max_invocations=3 ,priority=15,),
    ConditionalRequirement(internal_document_search, only_after=ThinkTool, consecutive_allowed=True, min_invocations=1, priority=20,),
  ]
elif pattern == 'Planning':
  requirement_set=[
    ConditionalRequirement(ThinkTool, force_at_step=1 ),
    ConditionalRequirement(wikipedia_tool, only_after=ThinkTool),
    ConditionalRequirement(DuckDuckGoSearchTool, only_after=wikipedia_tool),
    ConditionalRequirement(internal_document_search, only_after=DuckDuckGoSearchTool),
  ]
elif pattern == 'Unconstrained':
  requirement_set=[]
else:
  requirement_set=[]

## Explore Observability: See what is happening under the hood

Create the function that sets up observability using `OpenTelemetry` and [Arize's Phoenix Platform](https://arize.com/docs/phoenix/inferences/how-to-inferences/manage-the-app). There a several ways to view what is happening under the hood of your agent. View the observability documentation [here](https://framework.beeai.dev/modules/observability).

In [ ]:
def setup_observability(endpoint: str = "http://localhost:6006/v1/traces") -> None:
    """
    Sets up OpenTelemetry with OTLP HTTP exporter and instruments the beeai framework.
    """
    resource = Resource(attributes={})
    tracer_provider = trace_sdk.TracerProvider(resource=resource)
    tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
    trace_api.set_tracer_provider(tracer_provider)

    BeeAIInstrumentor().instrument()

load_dotenv()
# Enable OpenTelemetry integration
setup_observability("http://localhost:6006/v1/traces")
px_session = px.launch_app()

##  Assemble and Test your BeeAI Agent

This is the part we've been working towards! Let's assemble the agent with all the parts we created.

In [ ]:
agent = RequirementAgent(
    llm=llm,
    instructions= instruct_prompt,
    memory = memory,
    tools=[ThinkTool(), DuckDuckGoSearchTool(), wikipedia_tool, internal_document_search],
    requirements=requirement_set,
    # Log intermediate steps to the console
    middlewares=[GlobalTrajectoryMiddleware(included=[Tool])],
)

question = "I'm planning on meeting the Moderna rep at the next conference. Give me a one pager and remind me where we left off on previous internal discussions."

response = await agent.run(question, max_retries_per_step=3, total_max_retries=25)
print(response.last_message.text)

In [ ]:
px_session.view()